# 使用 Bedrock AgentCore Identity 保护凭证

本研讨会演示如何将 Strands Agents 与 Amazon Bedrock AgentCore Identity 集成，以在构建 AI 代理时安全地管理外部服务的 API 密钥和凭证。

## 概述

在本实验中，您将：
- 了解安全凭证管理挑战
- 理解 Bedrock AgentCore Identity 功能
- 为外部服务创建 API 密钥凭证提供程序
- 在 AI 代理中测试安全凭证检索
- 探索凭证管理的最佳实践

## 先决条件

开始本实验之前，请确保您具备：
- 已配置 AWS 凭证（IAM 角色或环境变量）
- 已安装所需的 Python 包
- 对 Strands Agents 和 Bedrock AgentCore 概念有基本了解
- 用于测试的外部 API 密钥（例如 Exa API 密钥）

如果您没有在假设 IAM 角色的环境中运行，请将您的 AWS 凭证设置为环境变量：

In [ ]:
import os

#os.environ["AWS_ACCESS_KEY_ID"]=<YOUR ACCESS KEY>
#os.environ["AWS_SECRET_ACCESS_KEY"]=<YOUR SECRET KEY>
#os.environ["AWS_SESSION_TOKEN"]=<OPTIONAL - YOUR SESSION TOKEN IF TEMP CREDENTIAL>
#os.environ["AWS_REGION"]=<AWS REGION WITH BEDROCK AGENTCORE AVAILABLE>

为 Strands Agents 和 Bedrock AgentCore Python SDK 安装所需的包：

In [2]:
#%pip install -q strands-agents strands-agents-tools bedrock-agentcore

## 获取 Exa API 密钥以连接远程 Exa MCP

在本实验中，我们将与 [远程 Exa MCP](https://docs.exa.ai/reference/exa-mcp) 交互，通过 Exa 搜索 API 执行实时网络搜索，这需要 Exa API 密钥来连接。

Exa MCP 服务器 URL：```https://mcp.exa.ai/mcp?exaApiKey=your-exa-api-key```

要获取 Exa API 密钥，请前往 [Exa 登录页面](https://dashboard.exa.ai/login) 使用您的电子邮件注册。

然后前往 Exa 仪表板中的 [API 密钥部分](https://dashboard.exa.ai/api-keys) 创建 API 密钥。将 API 密钥复制到下面代码中的 `EXA_API_KEY`...

## 了解凭证管理挑战

### 演示不安全的 API 密钥使用

让我们首先演示在没有适当凭证管理的情况下尝试使用外部服务（Exa 搜索）时会发生什么。这将显示硬编码或无效 API 密钥所带来的安全风险和身份验证失败。

In [18]:
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

# !-------- UPDATE THE EXA API KEY HERE  --------!
EXA_API_KEY = os.environ.get('EXA_API_KEY')

# Connect to the weather MCP server
print("\nConnecting to MCP Server...")
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))

with exa_server:
    # Combine all tools - they all work the same way!
    mcp_tools = (
        exa_server.list_tools_sync()
    )

    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")
    
    # Create agent with Exa MCP tools
    agent = Agent(model="us.amazon.nova-pro-v1:0",
                  system_prompt="You are a helpful assistant that provides concise responses.",
                  tools=mcp_tools)

    response = agent("介绍 Amazon Bedrock AgentCore是甚么? 包含哪些重要的功能?")
    print(response)


Connecting to MCP Server...
Available tools: ['web_search_exa', 'company_research_exa', 'crawling_exa', 'linkedin_search_exa', 'deep_researcher_start', 'deep_researcher_check']


<thinking> To answer the user's question about Amazon Bedrock AgentCore, I need to gather information about what it is and its key features. Since I don't have direct access to Amazon's internal documentation or specific product details that may not be publicly available, I will use the available tools to search for this information. The most appropriate tool for this task is `web_search_exa`, which can perform a web search to find relevant information about Amazon Bedrock AgentCore. </thinking>

Tool #1: web_search_exa
Amazon Bedrock AgentCore 是一种允许您安全地以大规模部署和操作高度功能强大的 AI 代理的服务。它提供了为动态代理工作负载而构建的基础设施、增强代理的强大工具以及实际部署所必需的控制。AgentCore 服务可以一起或独立使用，并与任何框架（包括 CrewAI、LangGraph、LlamaIndex 和 Strands Agents）以及 Amazon Bedrock 内部或外部的任何基础模型一起使用，从而为您提供最大的灵活性。AgentCore 消除了构建专门代理基础设施的繁重工作，使您能够加快代理投入生产的速度。

Amazon Bedrock AgentCore 的重要功能包括：

1. **部署代理**：使用 Amazon Bedrock 内部或外部的任何框架、协议或模型，以安全的方式部署和扩展动态 AI 代理。受益于完整的会话隔离和行业领先的对长时间运行工作负载（最长 8 小时）的支持，从而实现复杂的多步骤代理任务。通过与现有身份提供商的原生集成，跨服务无缝管理代理身份和访问，实现自动化身份验证和权

## 什么是 Bedrock AgentCore Identity？

Amazon Bedrock AgentCore Identity 为需要访问外部服务的 AI 代理提供安全的凭证管理。主要优势包括：

- **安全存储**：在 AWS Secrets Manager 中存储 API 密钥、令牌和凭证
- **运行时检索**：在运行时提供安全的凭证访问，无需硬编码
- **访问控制**：与 AWS IAM 集成，实现细粒度访问权限
- **审计跟踪**：维护凭证访问日志以进行安全监控
- **轮换支持**：支持自动凭证轮换和生命周期管理

此服务消除了在应用程序代码中硬编码敏感凭证的需要，降低了安全风险并改善了合规性。

## 创建安全凭证管理

### 步骤 1：创建 API 密钥凭证提供程序

现在我们将使用 Bedrock AgentCore Identity 创建一个 API 密钥凭证提供程序。这会将 Exa API 密钥安全地存储在 AWS Secrets Manager 中，并提供一种托管方式来访问它，而无需在代码中暴露凭证。

In [19]:
from bedrock_agentcore.services.identity import IdentityClient
import boto3

# !-------- UPDATE THE EXA API KEY HERE  --------!
EXA_API_KEY = os.environ.get('EXA_API_KEY') 

region = boto3.session.Session().region_name

#Configure API Key Provider
identity_client = IdentityClient(region=region)

try:
    api_key_provider = identity_client.create_api_key_credential_provider({
        "name": "exa-apikey-provider",
        "apiKey": EXA_API_KEY # Replace it with the API key you obtain from the external application vendor, e.g., OpenAI
    })
    print("Created AgentCore Identity API Key Credential Provider.")
    print(api_key_provider)
except Exception as e:
    print("AgentCore Identity API Key Credential Provider already exist.")

Created AgentCore Identity API Key Credential Provider.
{'ResponseMetadata': {'RequestId': '0c09228d-4413-478c-88a5-0ea37ed08352', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 26 Aug 2025 11:07:20 GMT', 'content-type': 'application/json', 'content-length': '326', 'connection': 'keep-alive', 'x-amzn-requestid': '0c09228d-4413-478c-88a5-0ea37ed08352', 'x-amzn-remapped-x-amzn-requestid': '8b5efb24-590d-4a07-8c0c-81b536fb1bc1', 'x-amzn-remapped-content-length': '326', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'P6RIaHNDIAMEZ4A=', 'x-amzn-trace-id': 'Root=1-68ad9568-760a1b285960a67b490baca0', 'x-amzn-remapped-date': 'Tue, 26 Aug 2025 11:07:20 GMT'}, 'RetryAttempts': 0}, 'apiKeySecretArn': {'secretArn': 'arn:aws:secretsmanager:us-east-1:710299592439:secret:bedrock-agentcore-identity!default/apikey/exa-apikey-provider-zQM2QJ'}, 'name': 'exa-apikey-provider', 'credentialProviderArn': 'arn:aws:bedrock-agentcore:us-east-1:710299592439:token-vault/default/apikeycredenti

### 步骤 2：测试安全凭证检索

现在让我们使用安全凭证检索来测试我们的代理。`@requires_api_key` 装饰器会在运行时自动从凭证提供程序检索 API 密钥，确保我们的代码中没有硬编码的秘密，同时保持安全最佳实践。

In [20]:
from bedrock_agentcore.identity.auth import requires_api_key
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client

@requires_api_key(provider_name="exa-apikey-provider")
def need_api_key(*, api_key: str):
    print(f'received api key for async func: {api_key}')
    return api_key

EXA_API_KEY = need_api_key()

# Connect to the EXA MCP server
print("\nConnecting to MCP Server...")
exa_server = MCPClient(lambda: streamablehttp_client(f"https://mcp.exa.ai/mcp?exaApiKey={EXA_API_KEY}"))

with exa_server:
    # Combine all tools - they all work the same way!
    mcp_tools = (
        exa_server.list_tools_sync()
    )

    print(f"Available tools: {[tool.tool_name for tool in mcp_tools]}")
    
    # Create agent with Exa MCP tools
    agent = Agent(model="us.amazon.nova-pro-v1:0",
                  system_prompt="You are a helpful assistant that provides concise responses.",
                  tools=mcp_tools)

    response = agent("介绍 Bedrock AgentCore 是甚么")
    print(response)

Created a workload identity
Created an user id
received api key for async func: e2152bee-f383-4ac8-a6b9-46d247234031

Connecting to MCP Server...
Available tools: ['web_search_exa', 'company_research_exa', 'crawling_exa', 'linkedin_search_exa', 'deep_researcher_start', 'deep_researcher_check']
<thinking> To introduce Bedrock AgentCore, I need to gather information about it. Since I don't have prior knowledge about Bedrock AgentCore, I will use the web_search_exa tool to search for information about it. </thinking>

Tool #1: web_search_exa
Bedrock AgentCore 是亚马逊推出的一项服务，旨在帮助开发者部署和运营高性能的 AI 代理，确保其在生产环境中的安全性、可靠性和可扩展性。它提供了专门为动态代理工作负载设计的基础设施，强大的工具来增强代理能力，以及必要的控制措施以确保可信的运营。AgentCore 服务可以单独或组合使用，并与任何框架（包括 CrewAI、LangGraph、LlamaIndex 和 Strands Agents）以及任何基础模型（无论是在 Amazon Bedrock 内部还是外部）兼容，提供了极大的灵活性。AgentCore 消除了构建专门代理基础设施的繁重工作，使您能够加速代理的上线。其主要功能包括：

1. **安全部署代理**：使用任何框架、协议或模型在 Amazon Bedrock 内部或外部安全地部署和扩展动态 AI 代理。受益于完全的会话隔离和行业领先的对长时间运行工作负载（最长 8 小时）的支持，使复杂的多步骤代理任务成为可能。通过与现有身份提供商的原生集成，跨服务无缝管理代理身份和

让我们检查代理循环的详细执行流程，以了解代理如何处理请求并生成响应：

In [21]:
print("Agent Loop Detail")
print("-----------------")

print(f"Agent Loop Length: {len(agent.messages)})")
print("\nUser-Assistant Conversation:")
for message in agent.messages:
    print(message)

Agent Loop Detail
-----------------
Agent Loop Length: 4)

User-Assistant Conversation:
{'role': 'user', 'content': [{'text': '介绍 Bedrock AgentCore 是甚么'}]}
{'role': 'assistant', 'content': [{'text': "<thinking> To introduce Bedrock AgentCore, I need to gather information about it. Since I don't have prior knowledge about Bedrock AgentCore, I will use the web_search_exa tool to search for information about it. </thinking>\n"}, {'toolUse': {'toolUseId': 'tooluse_EJwTd21TRqmy5-R2kuL4hQ', 'name': 'web_search_exa', 'input': {'query': 'Bedrock AgentCore'}}}]}
{'role': 'user', 'content': [{'toolResult': {'status': 'success', 'toolUseId': 'tooluse_EJwTd21TRqmy5-R2kuL4hQ', 'content': [{'text': '{\n  "requestId": "44150f9862c43244b651c3218b68b2bf",\n  "autopromptString": "Bedrock AgentCore",\n  "resolvedSearchType": "neural",\n  "results": [\n    {\n      "id": "https://aws.amazon.com/bedrock/agentcore/",\n      "title": "Amazon Bedrock AgentCore (Preview) - AWS",\n      "url": "https://aws.amaz

## 资源清理（可选）

清理已部署的资源：

In [22]:
import boto3

region = boto3.session.Session().region_name

agentcore_control_client = boto3.client('bedrock-agentcore-control', region_name=region)

try:
    print("Deleting AgentCore Identity...")
    agentcore_control_client.delete_api_key_credential_provider(name="exa-apikey-provider")
    print("✓ AgentCore Identity deletion initiated")
except Exception as e:
    print(f"❌ Error during cleanup: {e}")
    print("You may need to manually clean up some resources.")

Deleting AgentCore Identity...
✓ AgentCore Identity deletion initiated
